# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

# To establish connection and begin executing queries, need a session
try: 
    cluster = Cluster(['127.0.0.1']) 
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Setting KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
#query1="SELECT * FROM music_history_by_session_item WHERE session_id = 338 and item_in_session = 4"

# Creating table for Query 1
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS music_history_by_session_item (
        session_id int,
        item_in_session int,
        artist_name text,
        song_title text,
        song_length float,
        PRIMARY KEY (session_id, item_in_session)
    )
    """)
except Exception as e:
    print(e)


Table 3: users_by_song
PRIMARY KEY (song_title, user_id):
Reasoning: For this query, which asks for all users who listened to a specific song, song_title is the partition key, allowing efficient retrieval of all users associated with that song. The user_id serves as the clustering key, ensuring uniqueness and potentially allowing for additional sorting.                   

# Pimary key choice for table one: music_history_by_session_item
## PRIMARY KEY (session_id, item_in_session):
### Reasoning: The query asks for the artist, song title, and length based on sessionId and itemInSession. By making session_id the partition key and item_in_session the clustering key, the data is efficiently organized by session and then by the order of items within the session, which allows Cassandra to quickly retrieve the specific entry.

In [9]:
# Fetching data from csv file into table 1 (music_history_by_session_item)
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        # Insert data into music_history_by_session_item
        query1 = """
        INSERT INTO music_history_by_session_item (session_id, item_in_session, artist_name, song_title, song_length)
        VALUES (%s, %s, %s, %s, %s)
        """
        session.execute(query1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))


### Query 1: Selecting the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [11]:
## SELECT statement to verify the query 1

query1_b=query = "SELECT artist_name, song_title, song_length FROM music_history_by_session_item WHERE session_id = 338 AND item_in_session = 4"

try:
    rows = session.execute(query1_b)
    for row in rows:
        print(row.artist_name, row.song_title, row.song_length)
except Exception as e:
    print(e)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [ ]:
#"SELECT artist_name, song_title, first_name, last_name FROM music_history_by_user_session WHERE user_id = 10 AND session_id = 182"
# Create table for Query 2
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS music_history_by_user_session (
        user_id int,
        session_id int,
        item_in_session int,
        artist_name text,
        song_title text,
        first_name text,
        last_name text,
        PRIMARY KEY ((user_id, session_id), item_in_session)
    )
    """)
except Exception as e:
    print(e)


# Primary key Choice for table 2:  music_history_by_user_session
## PRIMARY KEY ((user_id, session_id), item_in_session):
### Reasoning: This query requires retrieving songs listened to by a specific user during a particular session. The partition key (user_id, session_id) groups the data by user and session, and the clustering key item_in_session ensures that the results are sorted by the sequence in which the items were played within the session.


In [13]:
# fetching data from the csv file into the table 2 (music_history_by_user_session)
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        # Insert data into music_history_by_user_session
        query2 = """
        INSERT INTO music_history_by_user_session (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        session.execute(query2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))


### Query 2: Selecting the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [14]:
## Select statement to erify query 2

query2_b = "SELECT artist_name, song_title, first_name, last_name FROM music_history_by_user_session WHERE user_id = 10 AND session_id = 182"

try:
    rows = session.execute(query2_b)
    for row in rows:
        print(row.artist_name, row.song_title, row.first_name, row.last_name)
except Exception as e:
    print(e)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [15]:
#query3="SELECT * FROM users_by_song WHERE song_title = 'All Hands Against His Own'"

# Create table for Query 3
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS users_by_song (
        song_title text,
        user_id int,
        first_name text,
        last_name text,
        PRIMARY KEY (song_title, user_id)
    )
    """)
except Exception as e:
    print(e)

                    

# Primary key choice for table 3: users_by_song
## PRIMARY KEY (song_title, user_id):
### Reasoning: For this query, which asks for all users who listened to a specific song, song_title is the partition key, allowing efficient retrieval of all users associated with that song. The user_id serves as the clustering key, ensuring uniqueness and potentially allowing for additional sorting.                   

In [16]:
# fetching data from the csv file into the table 3 (users_by_song)
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        # Insert data into users_by_song
        query3 = """
        INSERT INTO users_by_song (song_title, user_id, first_name, last_name)
        VALUES (%s, %s, %s, %s)
        """
        session.execute(query3, (line[9], int(line[10]), line[1], line[4]))

### Query 3: Selecting every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [22]:
## Select statement to veryfy query 3
query3_b = "SELECT first_name, last_name FROM users_by_song WHERE song_title = 'All Hands Against His Own'"


try:
    rows = session.execute(query3_b)
    for row in rows:
        print(row.first_name, row.last_name)
except Exception as e:
    print(e)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Beautified Output for Query 1

In [24]:
import pandas as pd

# For Query 1
query = "SELECT artist_name, song_title, song_length FROM music_history_by_session_item WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query)
    df = pd.DataFrame(list(rows), columns=['Artist', 'Song Title', 'Song Length'])  # Convert iterator to list
    print(df)
except Exception as e:
    print(e)


      Artist                       Song Title  Song Length
0  Faithless  Music Matters (Mark Knight Dub)   495.307312


### Beautified Output for Query 2

In [25]:

# For Query 2
query = "SELECT artist_name, song_title, first_name, last_name FROM music_history_by_user_session WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query)
    df = pd.DataFrame(list(rows), columns=['Artist', 'Song Title', 'First Name', 'Last Name'])  # Convert iterator to list
    print(df)
except Exception as e:
    print(e)


              Artist                                         Song Title  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  First Name Last Name  
0     Sylvie      Cruz  
1     Sylvie      Cruz  
2     Sylvie      Cruz  
3     Sylvie      Cruz  


### Beautified Output for Query 3

In [26]:
# For Query 3
query="SELECT first_name, last_name FROM users_by_song WHERE song_title = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
    df = pd.DataFrame(list(rows), columns=['First Name', 'Last Name'])  # Convert iterator to list
    print(df)
except Exception as e:
    print(e)




   First Name Last Name
0  Jacqueline     Lynch
1       Tegan    Levine
2        Sara   Johnson


### Drop the tables before closing out the sessions

In [27]:
## Dropping the table before closing out the sessions
query4 = "DROP TABLE music_history_by_session_item"
query5 = "DROP TABLE music_history_by_user_session"
query6 = "DROP TABLE users_by_song"

try:
    session.execute(query4)
    session.execute(query5)
    session.execute(query6)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [28]:
session.shutdown()
cluster.shutdown()